In [21]:
import nltk
import re
from nltk import pos_tag, word_tokenize, sent_tokenize
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')

def read_file(file_name):
    with open(file_name) as f:
        lines = f.readlines()
    return lines

path = './data/test_book'

text = read_file(path)

text = [x.strip() for x in text]
text = ' '.join(text)
text = re.sub(' +', lambda match: ' '+'Q' * (len(match.group(0))-1), text)                                             


tokenizer = nltk.data.load("tokenizers/punkt/english.pickle")
# tokenizer._realign_boundaries = True   

sentences = tokenizer.tokenize(text)
for sent in sentences:    sent =  re.sub('Q+', ' ', sent)   

words = nltk.tokenize.word_tokenize(text)
token_sentences = [word_tokenize(sent) for sent in sentences]

pos_sentences = [nltk.pos_tag(sent) for sent in token_sentences]
chunked_sentences = nltk.ne_chunk_sents(pos_sentences, binary=True)
from collections import defaultdict

# Create the defaultdict: ner_categories
ner_categories = defaultdict(int)
chunked_sentences = nltk.ne_chunk_sents(pos_sentences, binary=False)


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/louis/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/louis/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!


In [31]:
relevant_tags= ['PERSON', 'LOCATION']
predicted_labels =[]

from nltk.translate.ribes_score import position_of_ngram


chunked_sentences = nltk.ne_chunk_sents(pos_sentences, binary=False)


index = 0
# Create the nested for loop
for i,chuncked_sent in enumerate(chunked_sentences):
    raw_sent = sentences[i]
    for chunk in chuncked_sent:
        if hasattr(chunk, 'label'):
            if chunk.label() in relevant_tags:
                class_ = chunk.label()
                value = ' '.join(c[0] for c in chunk.leaves())
                pos = raw_sent.find(value)
                print(class_, value,  index+pos, text[index+pos:index+pos+len(value)])
                predicted_labels.append((class_, index+pos, index+pos+len(value)))
                if index == None:          
                    print(raw_sent)
                    print(class_, value, index)

    index += len(raw_sent)+1



PERSON V 9 V
PERSON Ned Land 425 Ned Land
PERSON Farragut 546 Farragut
PERSON Ned Land 575 Ned Land
PERSON Ned Land 879 Ned Land
PERSON Cape Vierges 1082 Cape Vierges
PERSON Cape Horn 1166 Cape Horn
PERSON Cape Horn 1657 Cape Horn
PERSON Capricorn 3576 Capricorn
PERSON Farragut 3815 Farragut
LOCATION Northern Pacific 4931 Northern Pacific
PERSON Captain Farragut 5989 Captain Farragut
PERSON Columbus 6010 Columbus
PERSON Conseil 8323 Conseil
PERSON Conseil 8323 Conseil
PERSON Conseil 8844 Conseil
PERSON Plantes 9011 Plantes
PERSON Conseil 9206 Conseil
PERSON Q 9617 Q
PERSON Ned 9764 Ned
PERSON Ned Land 10797 Ned Land
PERSON Conseil 10829 Conseil
PERSON Conseil 11045 Conseil
PERSON Ned Land 11096 Ned Land
PERSON Master Land 11152 Master Land
PERSON Ned Land 11942 Ned Land
PERSON Ned 11991 Ned
PERSON Conseil 12002 Conseil
PERSON Conseil 12286 Conseil
PERSON Conseil 13587 Conseil
PERSON Conseil 13875 Conseil
PERSON Ned Land 13902 Ned Land
PERSON Conseil 14701 Conseil
PERSON Ned Land 14751 

In [18]:
# read the annotated file
path = './data/job-1_annotations.json'
import json

def read_json(file_name):
    with open(file_name) as f:
        data = json.load(f)
    return data

file = read_json(path)

# for key in file['examples'][0].keys():
#     if key == 'content':
#         continue
#     print(key)
#     print(file['examples'][0][key])
#     print('\n')

annotations = file['examples'][0]['annotations']
# for annotation in annotations:
#     print(annotation['tag'])
#     print(annotation['value'])
#     print(annotation['start'])
#     print(annotation['end'])
#     print('\n')

# extract the true labels
true_labels = []
for annotation in annotations:
    true_labels.append((annotation['tag'], annotation['start'], annotation['end']))
    print(annotation['tag'], annotation['value'], annotation['start'], annotation['end'])

PERSON Ned Land 13902 13910
PERSON Conseil 12388 12395
LOCATION the
tropic of Cancer 4156 4176
LOCATION Cape Horn 1166 1175
PERSON Commander Farragut 7220 7238
PERSON the Canadian 20807 20819
PERSON the
captain of the _Monroe_ 383 410
PERSON Conseil 25329 25336
PERSON the Canadian 23001 23013
PERSON Hunger 18885 18891
PERSON Captain Nemo 27321 27333
PERSON Ned Land 15778 15786
PERSON Ned Land 9598 9606
PERSON Conseil 18987 18994
PERSON master 14031 14037
PERSON Ned Land 11096 11104
PERSON _Nautilus_ 10766 10776
PERSON _Nautilus_ 33826 33836
PERSON Master 11474 11480
LOCATION the tropic of Capricorn 3562 3585
PERSON Captain 26472 26479
PERSON Captain 25346 25353
LOCATION Island of Crespo 34958 34974
PERSON Ned Land 14751 14759
PERSON Ned 25276 25279
PERSON the Canadian 14654 14666
PERSON _Nautilus_ 24317 24327
PERSON Commander Farragut 536 554
PERSON M. Aronnax 19552 19562
LOCATION France 8783 8789
LOCATION the Sandwich Islands 4126 4146
PERSON Captain Farragut 5436 5452
LOCATION the is

In [33]:
# calculate the precision and recall

True_positives = 0
False_positives = 0
False_negatives = 0
for label in predicted_labels:
    if label in true_labels:
        True_positives += 1
    else:
        False_positives += 1
    
False_negatives = len(true_labels) - True_positives

precision = True_positives / (True_positives + False_positives)
recall = True_positives / (True_positives + False_negatives)

print('precision: ', precision)
print('recall: ', recall)
print('F1 score: ', 2 * precision * recall / (precision + recall))

precision:  0.7123287671232876
recall:  0.32298136645962733
F1 score:  0.4444444444444445
